In [ ]:
# Early Exit NLP

In [ ]:
!pip install --upgrade-strategy eager "optimum[openvino,nncf]"

In [ ]:
import torch
from optimum.intel import OVModelForQuestionAnswering
from transformers import AutoConfig, AutoTokenizer, pipeline, AutoModelForQuestionAnswering

In [ ]:
from ee_model.configuration_ee import EarlyExitConfig
from ee_model.modeling_ee import EarlyExitModelForQuestionAnswering

EarlyExitConfig.register_for_auto_class()
EarlyExitModelForQuestionAnswering.register_for_auto_class("AutoModelForQuestionAnswering")

In [ ]:
model_id = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_id)
config = EarlyExitConfig(base_model=model_id)
model = EarlyExitModelForQuestionAnswering(config)
model.set_entropy_threshold(10.0)
model.save_pretrained("ee_model_ckpts")
#model = OVModelForQuestionAnswering.from_pretrained("ee_model_ckpts", export=True)

questions = ["Which name is also used to describe the Amazon rainforest in English?", "Where do I live?"]
contexts =  ["The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain \"Amazonas\" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.", "My name is Clara and I live in Berkeley."]

answerer = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

for i in range(len(questions)):
    response = answerer(question=questions[i], context=contexts[i])
    print(response)